In [1]:
import numpy as np
import tensorflow.keras as K
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet import preprocess_input
from keras.optimizers import Adam

In [2]:
Edirectory = 'drive/MyDrive/Colab/project_Final/data/dataset/train'
Edirectorytest = 'drive/MyDrive/Colab/project_Final/data/dataset/test'
dataGen = K.preprocessing.image.ImageDataGenerator(rescale=1./255)
trainGen = dataGen.flow_from_directory(Edirectory,target_size=(32, 32))
testGen = dataGen.flow_from_directory(Edirectorytest,target_size=(32, 32))

Found 3400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [9]:
trainGen

## MY own NN

In [ ]:
model2 = K.models.Sequential()
model2.add(K.layers.Conv2D(128, 3, strides=(1, 1),input_shape=(32,32,3), activation='relu'))
model2.add(K.layers.Conv2D(256, 3, strides=(1, 1), activation='relu'))
model2.add(K.layers.MaxPooling2D(pool_size=(2,2)))
model2.add(K.layers.Conv2D(256, 3, strides=(1, 1), activation='relu'))
model2.add(K.layers.Conv2D(128, 3, strides=(1, 1),input_shape=(32,32,3), activation='relu'))
model2.add(K.layers.Conv2D(64, 3, strides=(1, 1),activation='relu'))
model2.add(K.layers.Flatten())
model2.add(K.layers.Dense(2,activation='relu'))

In [ ]:
model2.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 30, 30, 256)       7168      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 28, 28, 128)       295040    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 12, 12, 64)        73792     
_________________________________________________________________
flatten_7 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 18434     
Total params: 394,434
Trainable params: 394,434
Non-trainable params: 0
_______________________________________________

In [ ]:
model2.compile(loss='categorical_crossentropy',
              optimizer= Adam(learning_rate=0.01),
              metrics=['accuracy'])

In [ ]:
history2 = model2.fit(trainGen,batch_size=32,epochs=10,verbose=1)
model2.save('drive/MyDrive/Colab/projet_Final/src/SavedModel2.h5')

Epoch 1/10
107/107 [==============================] - 5s 45ms/step - loss: 8.3435 - accuracy: 0.4824
Epoch 2/10
107/107 [==============================] - 5s 47ms/step - loss: 8.3435 - accuracy: 0.4824
Epoch 3/10
107/107 [==============================] - 5s 50ms/step - loss: 8.3435 - accuracy: 0.4824
Epoch 4/10
107/107 [==============================] - 5s 49ms/step - loss: 8.3435 - accuracy: 0.4824
Epoch 5/10
107/107 [==============================] - 5s 49ms/step - loss: 8.3435 - accuracy: 0.4824
Epoch 6/10
107/107 [==============================] - 5s 47ms/step - loss: 8.3435 - accuracy: 0.4824
Epoch 7/10
107/107 [==============================] - 5s 46ms/step - loss: 8.3435 - accuracy: 0.4824
Epoch 8/10
107/107 [==============================] - 5s 49ms/step - loss: 8.3435 - accuracy: 0.4824
Epoch 9/10
107/107 [==============================] - 5s 48ms/step - loss: 8.3435 - accuracy: 0.4824
Epoch 10/10
107/107 [==============================] - 5s 49ms/step - loss: 8.3435 - accura

## Transfer Learning

In [5]:
new_shape = K.Input(shape=(32,32,3))

In [6]:
res_model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=new_shape,
)

94773248/94765736 [==============================] - 1s 0us/step


In [7]:
for layer in res_model.layers[:143]:
  layer.trainable = False

In [8]:
model = K.models.Sequential()
model.add(res_model)
model.add(K.layers.Flatten())
model.add(K.layers.Dense(2, activation='softmax'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 14,980,098
Non-trainable params: 8,611,712
_________________________________________________________________


In [ ]:
# checkpoint = K.callbacks.ModelCheckpoint(filepath='drive/MyDrive/Colab/projet_Final/src/SavedModel.h5',
#                                          monitor='val_acc',
#                                          mode='max',
#                                          save_best_only=True)

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer= K.optimizers.SGD(
    learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD"),
              metrics=['accuracy'])

In [ ]:
history = model.fit(trainGen,batch_size=16,epochs=10,
                    verbose=1, validation_data=testGen)

Epoch 1/10
